In [1]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
import math
import sklearn.preprocessing as preprocessing
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('Indicators.csv')
data.shape

(5656458, 6)

In [3]:
rel_columns_7 = ['Adolescent fertility rate (births per 1,000 women ages 15-19)',
 'Age dependency ratio (% of working-age population)',
 'Age dependency ratio, young (% of working-age population)',
 'Birth rate, crude (per 1,000 people)',
 'Fertility rate, total (births per woman)',
 'Life expectancy at birth, female (years)',
 'Life expectancy at birth, male (years)',
 'Life expectancy at birth, total (years)',
 'Mortality rate, adult, female (per 1,000 female adults)',
 'Mortality rate, infant (per 1,000 live births)',
 'Mortality rate, under-5 (per 1,000)',
 'Population, ages 0-14 (% of total)',
 'Population, ages 15-64 (% of total)',
 'Survival to age 65, female (% of cohort)',
 'Survival to age 65, male (% of cohort)',
 'Adjusted net enrolment rate, primary, both sexes (%)',
 'Adjusted net enrolment rate, primary, female (%)',
 'Gross enrolment ratio, secondary, both sexes (%)',
 'Gross enrolment ratio, secondary, female (%)',
 'Gross enrolment ratio, secondary, male (%)',
 'Lower secondary completion rate, both sexes (%)',
 'Net enrolment rate, secondary, both sexes (%)',
 'Net enrolment rate, secondary, female (%)',
 'Net enrolment rate, secondary, male (%)',
 'Primary completion rate, both sexes (%)',
 'Primary completion rate, female (%)',
 'Survival rate to the last grade of primary education, both sexes (%)',
 'Lower secondary completion rate, female (%)',
 'Lower secondary completion rate, male (%)',
 'Survival rate to the last grade of primary education, female (%)',
 'Survival rate to the last grade of primary education, male (%)',
 'Survival rate to Grade 5 of primary education, female (%)',
 'Youth literacy rate, population 15-24 years, both sexes (%)',
 'Youth literacy rate, population 15-24 years, male (%)',
 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
 'Wanted fertility rate (births per woman)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Improved sanitation facilities (% of population with access)',
 'Improved sanitation facilities, rural (% of rural population with access)',
 'Improved sanitation facilities, urban (% of urban population with access)',
 'Improved water source (% of population with access)',
 'Improved water source, rural (% of rural population with access)',
 'Lifetime risk of maternal death (%)',
 'Maternal mortality ratio (modeled estimate, per 100,000 live births)',
 'Mortality rate, infant, female (per 1,000 live births)',
 'Mortality rate, infant, male (per 1,000 live births)',
 'Mortality rate, neonatal (per 1,000 live births)',
 'Mortality rate, under-5, female (per 1,000 live births)',
 'Mortality rate, under-5, male (per 1,000 live births)',
 'Prevalence of anemia among children (% of children under 5)',
 'Prevalence of anemia among pregnant women (%)',
 'Renewable energy consumption (% of total final energy consumption)',
 'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)',
 'Cause of death, by non-communicable diseases (% of total)',
 'Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)',
 'Survey mean consumption or income per capita, total population (2005 PPP $ per day)']

In [4]:
for i in rel_columns_7:
    if len(data[data['IndicatorName']==i]) < 500:
        rel_columns_7.remove(i)

len(rel_columns_7)

54

In [5]:
rel_columns_8 = ['Adolescent fertility rate (births per 1,000 women ages 15-19)',
 'Birth rate, crude (per 1,000 people)',
 'Net enrolment rate, secondary, both sexes (%)',
 'Net enrolment rate, secondary, female (%)',
 'Net enrolment rate, secondary, male (%)',
 'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
 'Mortality rate, under-5, female (per 1,000 live births)',
 'Mortality rate, under-5, male (per 1,000 live births)',
 'Prevalence of anemia among children (% of children under 5)',
 'Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)',
 'Cause of death, by non-communicable diseases (% of total)',
 'Survey mean consumption or income per capita, bottom 40% of population (2005 PPP $ per day)',
 'Survey mean consumption or income per capita, total population (2005 PPP $ per day)']

In [6]:
# 여기서부터 관련도 높은 feature 뽑는 것
target_id = 'Adolescent fertility rate (births per 1,000 women ages 15-19)';

target = data[data['IndicatorName']==target_id]

In [7]:
merged = data[data['IndicatorName']==target_id]

for i in rel_columns_8:
    target = data[data['IndicatorName']==i]
    target.rename(columns={"Value": i}, inplace = True)
    target=target.drop(columns=['IndicatorCode', 'IndicatorName', 'CountryName'])
#    print(target.head())
    merged_bef = merged
    merged = pd.merge(merged, target)
    if len(merged) < 500:
        merged = merged_bef
#    print(merged.head())

merged

c:\users\parksoyoung\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)","Birth rate, crude (per 1,000 people)","Net enrolment rate, secondary, both sexes (%)","Net enrolment rate, secondary, female (%)","Net enrolment rate, secondary, male (%)",Prevalence of anemia among children (% of children under 5)
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,69.020564,69.020564,35.138300,44.576275,39.293392,49.667217,53.673302
1,Caribbean small states,CSS,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,90.664634,90.664634,25.257613,64.583374,66.755806,62.447144,42.518994
2,Euro area,EMU,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,14.604634,14.604634,11.720194,79.480003,80.562958,78.446426,19.434867
3,European Union,EUU,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,22.283391,22.283391,12.375231,80.662552,81.711151,79.662552,22.427980
4,High income,HIC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,33.709503,33.709503,14.585758,81.518730,82.150581,80.926880,23.921785
5,High income: OECD,OEC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,26.662798,26.662798,13.652285,83.069412,83.782578,82.389725,18.861448
6,Least developed countries: UN classification,LDC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,135.511948,135.511948,42.473446,15.394602,12.155324,18.571140,75.306228
7,Middle East & North Africa (all income levels),MEA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,68.680675,68.680675,33.544956,49.202000,44.690079,53.519825,48.646734
8,Middle East & North Africa (developing only),MNA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,70.065273,70.065273,33.850139,49.028133,44.022461,53.780590,48.743728
9,North America,NAC,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,53.848894,53.848894,16.529283,83.866615,84.452255,83.308624,16.563037


In [8]:
merged = data[data['IndicatorName']==target_id]

for i in rel_columns_7:
    target = data[data['IndicatorName']==i]
    target.rename(columns={"Value": i}, inplace = True)
    target=target.drop(columns=['IndicatorCode', 'IndicatorName', 'CountryName'])
#    print(target.head())
    merged_bef = merged
    merged = pd.merge(merged, target)
    if len(merged) < 200:
        merged = merged_bef
#    print(merged.head())

merged

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, young (% of working-age population)","Birth rate, crude (per 1,000 people)",...,"Improved sanitation facilities, rural (% of rural population with access)","Improved sanitation facilities, urban (% of urban population with access)",Improved water source (% of population with access),"Improved water source, rural (% of rural population with access)",Lifetime risk of maternal death (%),"Maternal mortality ratio (modeled estimate, per 100,000 live births)","Mortality rate, neonatal (per 1,000 live births)",Prevalence of anemia among children (% of children under 5),Prevalence of anemia among pregnant women (%),Renewable energy consumption (% of total final energy consumption)
0,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,102.0524,102.0524,89.996571,85.008764,34.509,...,22.9,62.0,92.2,86.9,1.196932,243.0,25.9,47.0,40.5,47.584663
1,Guinea,GIN,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,180.7146,180.7146,89.617082,83.059669,46.505,...,4.7,17.6,52.4,39.1,7.033457,1040.0,62.9,82.9,63.6,92.593984
2,Niger,NER,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,222.2130,222.2130,100.023714,95.353577,55.122,...,1.2,18.9,33.7,28.8,6.785329,873.0,54.5,87.2,63.0,86.715259
3,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,93.6652,93.6652,88.163978,83.176254,33.498,...,23.8,62.7,92.8,87.1,1.099415,233.0,25.7,46.0,39.8,48.082742
4,Costa Rica,CRI,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,92.4632,92.4632,65.942106,58.011655,25.984,...,83.1,94.2,93.1,86.7,0.160381,45.0,8.8,27.5,32.2,44.695587
5,Niger,NER,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,222.3240,222.3240,100.616474,95.874054,55.106,...,1.4,19.7,34.7,29.6,6.715623,858.0,53.6,87.3,63.1,87.444913
6,Togo,TGO,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,113.1332,113.1332,96.986658,91.123519,42.056,...,6.8,23.4,48.1,35.2,3.484661,573.0,42.6,81.2,62.8,82.087190
7,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1992,85.2780,85.2780,86.125346,81.167026,32.567,...,24.7,63.4,93.1,87.3,1.022598,227.0,25.4,45.1,39.1,44.708291
8,"Congo, Dem. Rep.",ZAR,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1992,133.3220,133.3220,96.098081,90.377191,47.713,...,14.4,30.0,44.2,24.7,5.944522,883.0,41.0,79.9,57.7,95.875225
9,Mozambique,MOZ,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1992,188.9320,188.9320,96.884321,90.704831,45.768,...,2.2,34.4,35.6,22.8,7.953500,1290.0,59.6,79.1,53.0,93.118688


In [9]:
# 기준: value 등급? 만듦
print(min(merged['Value']))
print(max(merged['Value']))
print(merged['Value'].mean())

2.093
222.324
72.10240748663102


In [10]:
merged['Value']/100

0      1.020524
1      1.807146
2      2.222130
3      0.936652
4      0.924632
         ...   
369    0.526508
370    0.289356
371    0.186794
372    0.332360
373    0.812292
Name: Value, Length: 374, dtype: float64

In [11]:
for i in range(len(merged)):
    print(sigmoid(merged.iloc[i]['Value']))
    
merged

NameError: name 'sigmoid' is not defined

In [12]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [13]:
list(merged['Value'])

[102.0524,
 180.7146,
 222.213,
 93.6652,
 92.4632,
 222.324,
 113.1332,
 85.27799999999999,
 133.322,
 188.93200000000002,
 164.1812,
 82.6834,
 132.6888,
 66.9284,
 187.2686,
 80.0888,
 132.0556,
 97.5668,
 185.6052,
 36.0614,
 18.6414,
 53.9528,
 183.9418,
 70.3002,
 33.2256,
 16.7552,
 14.869000000000002,
 86.62100000000001,
 220.90400000000002,
 27.554000000000002,
 87.655,
 69.9218,
 22.265,
 2.8014,
 36.3276,
 88.656,
 12.2498,
 50.023999999999994,
 107.9878,
 50.8622,
 100.2308,
 83.273,
 67.5386,
 61.8732,
 10.7228,
 110.439,
 83.5932,
 27.6284,
 112.6048,
 109.9886,
 65.2928,
 2.6508,
 82.053,
 92.2484,
 36.3662,
 183.3266,
 86.92200000000001,
 6.5344,
 11.6816,
 41.8544,
 106.5926,
 89.5158,
 78.891,
 65.1554,
 212.41,
 58.1128,
 9.5572,
 83.2328,
 26.2586,
 81.2662,
 108.0704,
 2.5002,
 79.76899999999999,
 91.3626,
 36.4048,
 79.6914,
 184.6824,
 118.0082,
 11.1134,
 105.1974,
 62.7722,
 138.203,
 54.3524,
 86.414,
 24.8888,
 79.3116,
 106.1522,
 6.8839999999999995,
 84.795

In [14]:
merged['grade'] = merged['Value']/50
for i in range(len(merged)):
    merged.loc[i, 'grade'] = math.ceil(merged.loc[i, 'grade'])
merged

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, young (% of working-age population)","Birth rate, crude (per 1,000 people)",...,"Improved sanitation facilities, urban (% of urban population with access)",Improved water source (% of population with access),"Improved water source, rural (% of rural population with access)",Lifetime risk of maternal death (%),"Maternal mortality ratio (modeled estimate, per 100,000 live births)","Mortality rate, neonatal (per 1,000 live births)",Prevalence of anemia among children (% of children under 5),Prevalence of anemia among pregnant women (%),Renewable energy consumption (% of total final energy consumption),grade
0,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,102.0524,102.0524,89.996571,85.008764,34.509,...,62.0,92.2,86.9,1.196932,243.0,25.9,47.0,40.5,47.584663,3.0
1,Guinea,GIN,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,180.7146,180.7146,89.617082,83.059669,46.505,...,17.6,52.4,39.1,7.033457,1040.0,62.9,82.9,63.6,92.593984,4.0
2,Niger,NER,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,222.2130,222.2130,100.023714,95.353577,55.122,...,18.9,33.7,28.8,6.785329,873.0,54.5,87.2,63.0,86.715259,5.0
3,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,93.6652,93.6652,88.163978,83.176254,33.498,...,62.7,92.8,87.1,1.099415,233.0,25.7,46.0,39.8,48.082742,2.0
4,Costa Rica,CRI,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,92.4632,92.4632,65.942106,58.011655,25.984,...,94.2,93.1,86.7,0.160381,45.0,8.8,27.5,32.2,44.695587,2.0
5,Niger,NER,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,222.3240,222.3240,100.616474,95.874054,55.106,...,19.7,34.7,29.6,6.715623,858.0,53.6,87.3,63.1,87.444913,5.0
6,Togo,TGO,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,113.1332,113.1332,96.986658,91.123519,42.056,...,23.4,48.1,35.2,3.484661,573.0,42.6,81.2,62.8,82.087190,3.0
7,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1992,85.2780,85.2780,86.125346,81.167026,32.567,...,63.4,93.1,87.3,1.022598,227.0,25.4,45.1,39.1,44.708291,2.0
8,"Congo, Dem. Rep.",ZAR,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1992,133.3220,133.3220,96.098081,90.377191,47.713,...,30.0,44.2,24.7,5.944522,883.0,41.0,79.9,57.7,95.875225,3.0
9,Mozambique,MOZ,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1992,188.9320,188.9320,96.884321,90.704831,45.768,...,34.4,35.6,22.8,7.953500,1290.0,59.6,79.1,53.0,93.118688,4.0


In [15]:
merged['grade'].unique()

array([3., 4., 5., 2., 1.])

In [16]:
merged['grade'].value_counts()

2.0    172
1.0    128
3.0     35
4.0     27
5.0     12
Name: grade, dtype: int64

In [17]:
print(min(merged['grade']))
print(max(merged['grade']))
print(merged['grade'].mean())

1.0
5.0
1.9919786096256684


In [18]:
# label encode categorical data
def labelEncode():
    for i in merged.columns:
        le = preprocessing.LabelEncoder()
        le.fit(merged[i])
        merged[i]= le.transform(merged[i])

In [19]:
# label encode categorical data (array)
def labelEncode_arr(y):
    '''
    label encoding array
    return value: label encoded array
    '''
    le = preprocessing.LabelEncoder()
    le.fit(y)
    y= le.transform(y)
    return y

In [20]:
# scaling data (MinMaxScaler)
def minmax(X):
    scaler = preprocessing.MinMaxScaler()
    fit = scaler.fit(X)

    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=list(X.index.values))
    return X_scaled

In [21]:
# scaling data (MinMaxScaler)
def standard(X):
    scaler = preprocessing.StandardScaler()
    fit = scaler.fit(X)

    X_scaled = scaler.transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=list(X.index.values))
    return X_scaled


In [22]:
# cleansing dataset
def cleansing(X):
    # cleansing missing data
    X.dropna(axis=0, thresh=1, inplace=True)
    # cleansing outliers
    robust = preprocessing.RobustScaler().fit(X)

    X_robust = robust.transform(X)
    X_robust = pd.DataFrame(X, columns=X.columns, index=list(X.index.values))
    return X_robust

In [23]:
eps=[0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5 ]
min_samples=[ 3, 5, 10, 15, 20, 30, 50, 100 ]
dis_measure = ['euclidean', 'hamming']

In [33]:
eps=[10, 20, 30, 40, 50 ]
min_samples=[ 3, 5, 10, 15, 20, 30, 50, 100 ]
dis_measure = ['euclidean', 'hamming']

In [25]:
def purity_score(y, y_label):
    print('y', y)
    print('label', y_label)
    unique_labels=set(y_label)
    
    sum_=0
    
    purity = pd.DataFrame(0, index=np.arange(len(unique_labels)), columns=["0", "1"])
#    print(purity)
    for i in range(len(y_label)):
        if y[i] ==0:
            purity.iloc[y_label[i], 0] = purity.iloc[y_label[i], 0]+1
        else:
            purity.iloc[y_label[i], 1] = purity.iloc[y_label[i], 1]+1

    for j in range(len(purity)):
        if purity.iloc[j, 0] > purity.iloc[j, 1]:
            sum_=sum_ + purity.iloc[j, 0]
        else:
            sum_=sum_ + purity.iloc[j, 1]
    
    return sum_/len(y)

In [26]:
def none(i, j, n):
    X=merged.drop(columns=['grade'])
    y=merged['grade']
    X = cleansing(X)
    db = DBSCAN(eps = i, min_samples = j, metric= n).fit(X) 

    cluster_labels = db.labels_
    n_clusters = len(set(cluster_labels))
    
    print('clusters', n_clusters)
    print(purity_score(y, cluster_labels))

In [27]:
merged.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value,"Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, young (% of working-age population)","Birth rate, crude (per 1,000 people)",...,"Improved sanitation facilities, urban (% of urban population with access)",Improved water source (% of population with access),"Improved water source, rural (% of rural population with access)",Lifetime risk of maternal death (%),"Maternal mortality ratio (modeled estimate, per 100,000 live births)","Mortality rate, neonatal (per 1,000 live births)",Prevalence of anemia among children (% of children under 5),Prevalence of anemia among pregnant women (%),Renewable energy consumption (% of total final energy consumption),grade
0,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,102.0524,102.0524,89.996571,85.008764,34.509,...,62.0,92.2,86.9,1.196932,243.0,25.9,47.0,40.5,47.584663,3.0
1,Guinea,GIN,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,180.7146,180.7146,89.617082,83.059669,46.505,...,17.6,52.4,39.1,7.033457,1040.0,62.9,82.9,63.6,92.593984,4.0
2,Niger,NER,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1990,222.2130,222.2130,100.023714,95.353577,55.122,...,18.9,33.7,28.8,6.785329,873.0,54.5,87.2,63.0,86.715259,5.0
3,Botswana,BWA,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,93.6652,93.6652,88.163978,83.176254,33.498,...,62.7,92.8,87.1,1.099415,233.0,25.7,46.0,39.8,48.082742,2.0
4,Costa Rica,CRI,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1991,92.4632,92.4632,65.942106,58.011655,25.984,...,94.2,93.1,86.7,0.160381,45.0,8.8,27.5,32.2,44.695587,2.0


In [28]:
merged = merged.drop(columns=['CountryCode', 'IndicatorCode'])

In [29]:
def labeling(i, j, n):
    merged['CountryName']= labelEncode_arr(merged['CountryName'])
    merged['IndicatorName']= labelEncode_arr(merged['IndicatorName'])
    
    X=merged.drop(columns=['grade'])
    y=merged['grade']

    X = cleansing(X)

    db = DBSCAN(eps = i, min_samples = j, metric= n).fit(X) 

    cluster_labels = db.labels_
    n_clusters = len(set(cluster_labels))
    
    print('clusters', n_clusters)
    print(purity_score(y, cluster_labels))

In [30]:
def label_minmax(i, j, n):
    merged['CountryName']= labelEncode_arr(merged['CountryName'])
    merged['IndicatorName']= labelEncode_arr(merged['IndicatorName'])

    X=merged.drop(columns=['grade'])
    y=merged['grade']

    X_scaled = minmax(X)
    X_scaled = cleansing(X_scaled)
    
    db = DBSCAN(eps = i, min_samples = j, metric= n).fit(X_scaled) 

    cluster_labels = db.labels_
    n_clusters = len(set(cluster_labels))
    
    print('clusters', n_clusters)
    print(purity_score(y, cluster_labels))

In [31]:
def label_standard(i, j, n):
    merged['CountryName']= labelEncode_arr(merged['CountryName'])
    merged['IndicatorName']= labelEncode_arr(merged['IndicatorName'])

    X=merged.drop(columns=['grade'])
    y=merged['grade']

    X_scaled = standard(X)
    X_scaled = cleansing(X_scaled)
    
    db = DBSCAN(eps = i, min_samples = j, metric= n).fit(X_scaled) 

    cluster_labels = db.labels_
    n_clusters = len(set(cluster_labels))
    
    print('clusters', n_clusters)
    print(purity_score(y, cluster_labels))

In [ ]:
for i in eps:
    for j in min_samples:
        for n in dis_measure:
            print(i, j, n)
            # varying preprocessing            

            # none
            if n == 'hamming':
                print('none')
                none(i, j, n)
            
            # labelEncoding
            print('\nlabelEncoding')
            labeling(i, j, n)
            
            # labelEncoding + MinMaxScaler
            print('\nlabelEncoding + MinMaxScaler')
            label_minmax(i, j, n)
            
            # labelEncoding + StandardScaler
            print('\nlabelEncoding + StandardScaler')
            label_standard(i, j, n)
            print('\n')


10 3 euclidean

labelEncoding
clusters 11
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1
 -1 -1 -1  2 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  2
 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1  1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1
 -1  0 -1 -1 -1 -1 -1  3 -1 -1 -1  4  2 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  4  2 -1 -1  6 -1  0 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1  4  5  2 -1 -1 -1  6 -1 -1  0 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  3 -1 -1 -1 -1 -1 -1  4  5  2 -1 -1 -1  6
 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  7  3 -1 -1 -1 -1  5  2
 -1 -1 -1 -1  6 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1  7 -1 -1 -1 -1  4  5
  2 -1  8 -1 -1  6 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 

1.0

labelEncoding + MinMaxScaler
clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0

labelEncoding + Stan

clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0


10 10 hamming
none
clusters 1
y 0      3.0
1      4.0

clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0

labelEncoding + MinMaxScaler
clusters 1
y 0      3.0
1

1.0

labelEncoding + StandardScaler
clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0


10 30 euclidean



...

clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0

labelEncoding + StandardScaler
clusters 1
y 0      3.0

1.0

labelEncoding
clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0

labelEncoding + MinMaxScaler
cluste

clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0


50 10 euclidean

labelEncoding
clusters 4
y 0      3.

1.0

labelEncoding + MinMaxScaler
clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0

labelEncoding + Stan

1.0


50 20 hamming
none
clusters 1
y 0      3.0
1      4.0
2      5.0
3      2.0
4      2.0
      ... 
369    2.0
370    1.0
371    1.0
372    1.0
373    2.0
Name: grade, Length: 374, dtype: float64
label [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
1.0

labelEncoding
clusters 1
y 0 